# I have implemented RAG based on similarty on preprocessed CSV dataset.

**NOTE:To view the results directly: Scroll to the bottom of the Colab file.**


In [1]:
# prompt: mount gdrive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q -U langchain langchain-community
!pip install -q -U langchain-groq
!pip install -q -U chromadb
!pip install -q -U sentence-transformers==2.7.0
!pip install -q -U langchain-core
!pip install -q -U langchain
!pip install -q -U faiss-gpu==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.7 MB/s eta 0:00:0

# **Here I have loaded preprocesses csv data which I have converted into embeddings before storing into database**


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
# Load the CSV file
loader = CSVLoader(file_path='/content/drive/MyDrive/1rag project/final_processed.csv')
data = loader.load()

# Split the documents into smaller chunks
#separator=""
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs  = text_splitter.split_documents(data)

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

<ipython-input-9-84afd9193cb7>:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-9-84afd9193cb7>:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **there are 17135 rows for FORD AND TOYOTA so to generate embeddings it will take some time**

In [ ]:
print(len(chunked_docs))

17135


no need to run everytime


In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


THE ABOVE PROCESS WILL TAKE LOTS OF TIME SO MAKE SURE TO SAVE IT ONCE IT GETS CONVERTED

In [3]:
from langchain.vectorstores import FAISS

db.save_local('/content/drive/MyDrive/1rag_project/faiss_index')


NameError: name 'db' is not defined

In [4]:
import pickle
with open('/content/drive/MyDrive/1rag_project/faiss_index.pkl', 'wb') as f:
    pickle.dump(db, f)

NameError: name 'db' is not defined

# reinitialize the db with same embedding model


In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import faiss
import pickle


embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')


index_path = '/content/drive/MyDrive/1rag_project/faiss_index/index.faiss'
metadata_path = '/content/drive/MyDrive/1rag_project/faiss_index/index.pkl'


faiss_index = faiss.read_index(index_path)

# Load the metadata (e.g., docstore and index_to_docstore_id)
with open(metadata_path, 'rb') as f:
    docstore, index_to_docstore_id = pickle.load(f)

# Reconstruct the FAISS object
db = FAISS(
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_function
)


<ipython-input-5-e4b07089975d>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Here the data is very large so you can increase value of k in retriever

In [6]:

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 5}
)

In [7]:

query = "TAURUS"
docs = retriever.get_relevant_documents(query)

for doc in docs:
    print(doc.page_content)

<ipython-input-7-3fce658dc043>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


Co_Name: ford
Mo_Name: TAURUS
Year: 1987
row_name: 22848|94E036000|FORD|TAURUS|1987
ManufacturerContact: THE ARC WELDS ON THE SWAY BAR BRACKET, WHERE IT ATTACHES TO THE STRUT RESERVE TUBE, DID NOT FUSE PROPERLY.
MfgReportDate: OVER TIME, A PORTION OF THE BRACKET CAN TEAR AWAY FROM THE STRUT, FALLING INTO AND PUNCTURING THE TIRE OR CAUSING DAMAGE TO THE WHEEL, WHICH COULD RESULT IN AN ACCIDENT.
FMVSS: MONROE DEALERS WILL REPLACE THE STRUT FREE OF CHARGE.
DescOfDefect: EQUIPMENT DESCRIPTION: AFTERMARKET SHOCK ABSORBERS FOR 1986 - 1994 FORD TAURUS AND MERCURY SABLE PASSENGER CARS; PART 71780, 81780.IF THE VEHICLE IS PRESENTED TO AN AUTHORIZED DEALER ON AN AGREED UPON SERVICE DATE AND THE REMEDY IS NOT PROVIDED FREE OF CHARGE WITHIN A REASONABLE TIME OR THE REMEDY DOES NOT CORRECT THE DEFECT, PLEASE CONTACT MONROE AT 1-800-521-6754. ALSO, CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFETY ADMINISTRATION'S AUTO SAFETY HOTLINE AT 1-800-424-9393.
Co_Name: ford
Mo_Name: TAURUS
Year: 1992
row_name: 22

In [8]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough



In [9]:
template = """User: You are an AI Assistant that answer question based on context in which you have to generate a summary based on the issue or recall .
CONTEXT: {context}
Query: {question}
only return the summary no need to return the acctual rows and coloumns of database because you have to generate output in a paragraph manner which is easy to understand
Assistant:"""

In [10]:
prompt=ChatPromptTemplate.from_template(template)

# MY GPU CREDITS GOT EXHAUSTED IN COLAB SO I AM USING GROQ BUI CHECKOUT MY GITHUB REPO THERE I HAVE UPLOADED ANOTHER REPO TO RUN LLM LOCALLY


In [11]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [13]:
from langchain_groq import ChatGroq
chat=ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=1,
    max_tokens=1000,
    # top_p=1,
    stream=True,
    stop=None,
)

/usr/local/lib/python3.10/dist-packages/langchain_groq/chat_models.py:362: UserWarning: WARNING! stream is not default parameter.
                    stream was transferred to model_kwargs.
                    Please confirm that stream is what you intended.
  warnings.warn(


In [14]:
output_parser=StrOutputParser()

In [15]:
chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | chat
    | output_parser
)

In [17]:
query="Cruise Control System is faulty in which model"
for chunk in chain.stream(query):
    print(chunk, end="",flush=True)

Based on the documents provided, there are several model numbers for Toyota vehicles that have faulty Cruise Control Systems. 

The 1982 Toyota Cressida and Toyota Celica models have a potential issue with the Cruise Control system due to cracks in the soldered terminals of the computer. This can cause the engine speed to instantly race and the vehicle to suddenly accelerate, leading to an accident.

The 1996 Toyota T100 and Toyota Celica models have failing Cruise Control systems that can accelerate above the intended set speed, potentially leading to an increase in the potential for a vehicle accident.

Lastly, the 1984 Toyota Corolla model has a malfunctioning Cruise Control computer that can occur upon shutting off the control after continuous use followed by cold ambient temperatures. This can result in engine racing if the engine is restarted with the main control switch in the 'on' position.

In all these instances, Toyota has issued recalls or warnings, and owners are advised t

In [20]:
query="what are the issue with ford focus"
for chunk in chain.stream(query):
    print(chunk, end="",flush=True)

Based on the provided context, it appears that there have been several issues with the Ford Focus model over the years. 

One of the issues was with the Fuel Delivery Module (FDM) in certain Ford Focus models from 2014, which may cause low fuel pressure and lead to a vehicle stall, increasing the risk of a crash. 

Another issue was with the clutch in certain Ford Focus and Fusion models from 2013 to 2016, equipped with a 1.6L or 1.0L GTDI engine and a 6-speed manual transmission. The clutch may fracture, causing damage to the transmission assembly and potentially leading to a transmission fluid leak, which can increase the risk of a fire.

The Fuel Delivery Module issue also affected Ford Focus models from 2000 and 2001, where contamination of the filter in the FDM can reduce and eventually block the flow of fuel to the engine, leading to engine hesitation, loss of power, and other symptoms. This can cause the engine to stall, which can result in a crash.

# the normal rag dose not have overall picture of the data so it cannot answer general question base on overall picture of data

In [22]:
query="which manufacturer have more issue ford or toyoya"
for chunk in chain.stream(query):
    print(chunk, end="",flush=True)

Based on the provided context, it appears that Ford has recalled several models due to a potential issue with air bag inflators. The recalls include model years 2005-2008 Ford GT, 2006-2011 Ford Fusion, 2005-2011 Ford Mustang, and 2007-2011 Ford Ranger, among others.

To answer the query, I did not find any information about Toyota in the provided context. However, since Toyota is not mentioned at all, we can conclude that Ford has more issues mentioned in the recall notices. The problems with Ford's air bag inflators may result in serious injury or death, and owners are advised not to drive their vehicles until the repair has been performed.

Summary: Ford has several models under recall due to a potential issue with air bag inflators, while Toyota is not mentioned in the provided context. Therefore, it can be concluded that Ford has more issues mentioned in the recall notices.

SINCE IN NORMAL RAG YOU HAVE TO USE RETRIEVER BASED ON SIMILARITY AND SET THE VALUE OF K SO IT WILL JUST RETRIEVE SOME DATA FROM DATASET BUT IT WOULD NOT HAVE OVERALL PICTURE OF COMPLETE DATA SO FOR THAT GRAPH RAG CAN BE USED

PLEASE CHECKOUT GRAPH RAG IMPLEMENTATION ON SAME GITHUB REPO IF YOU FIND IT INTRESTING TILL NOW